<a href="https://colab.research.google.com/github/BalaLearning/practical-python-for-algorithmic-trading-4403633/blob/main/StockPrediction_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import tensorflow as tf
START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

# Define a function to load the dataset

def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

    data = load_data('AAPL')
    df=data
    df.head()

        # Inspect the columns of the DataFrame to identify the correct columns to drop
    print(df.columns)

    # Sort the columns (MultiIndex) to ensure correct dropping
    df.sort_index(axis=1, inplace=True)
    print("\nColumns after sorting:")
    print(df.columns)

    # Drop the 'Date' column from the MultiIndex
    df.drop('Date', axis=1, inplace=True)

    df.head()

    plt.plot(df.Close)

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0,1))
    df = scaler.fit_transform(df)

    # Splitting data into training and testing sets
    training_size = int(len(df) * 0.65)
    test_size = len(df) - training_size
    train_data, test_data = df[0:training_size,:], df[training_size:len(df),:1]

    # Function to create dataset for LSTM
    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2..,99   100
            dataX.append(a)
            dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)

    # Reshape into X=t,t+1,t+2..t+99 and Y=t+100
    time_step = 100
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    # Reshape input to be [samples, time steps, features]
    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

    # Create the LSTM model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import LSTM

    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')

    model.summary()

    model.fit(X_train,y_train,validation_split=0.1,epochs=60,batch_size=64,verbose=1)

    # Make predictions
    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)

    # Inverse transform the predictions
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)

    # Evaluate the model
    import math
    from sklearn.metrics import mean_squared_error

    math.sqrt(mean_squared_error(y_train,train_predict))

    math.sqrt(mean_squared_error(y_test,test_predict))

    # Plotting
    # shift train predictions for plotting
    look_back=100
    trainPredictPlot = np.empty_like(df)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(df)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(df)-1, :] = test_predict
    # plot baseline and predictions
    plt.plot(scaler.inverse_transform(df))
    plt.plot(trainPredictPlot)
    plt.plot(testPredictPlot)
    plt.show()